# Subsegmentation of ALIC tracts

This notebook is used for rapid prototyping of the ALIC segmentation pipeline. Once complete and relatively stable, it will be converted to a python script.

## Setup

In [ ]:
%matplotlib inline

In [ ]:
import os
import sys
sys.path.append('wma_pyTools')

# import wmaPyTools.roiTools

In [ ]:
import os
import sys
import itertools
import numpy as np
from warnings import warn
from pathlib import Path
import nibabel as nib
from scipy import ndimage
#import random

#make sure that wma_pyTools is right in the working directory, or that
#the package can otherwise be imported effectively
#sys.path.append('wma_pyTools')
startDir=Path(os.getcwd())
import pandas as pd
import wmaPyTools.roiTools
import wmaPyTools.analysisTools
import wmaPyTools.segmentationTools
import wmaPyTools.streamlineTools
import wmaPyTools.visTools

#dipy
from dipy.tracking.utils import reduce_labels
from dipy.tracking import utils
import dipy.io.streamline
from dipy.tracking.utils import density_map

In [ ]:
# extract and organize inputs



In [ ]:
# coregister B0 image into acpc space


In [ ]:
# define expected inputs and check for them
data_dir = startDir /'indata'

parcellationPath = data_dir / 'aparc+aseg.nii.gz'
refT1Path = data_dir / 'T1w_acpc_dc_restore.nii.gz'
diffPath = data_dir / 'eddy_wrapped_avg_image_space-acpc.nii.gz'
diff_b0acpc_path = data_dir / 'eddy_wrapped_B0_image_space-acpc.nii.gz'
diff_unreg_path = data_dir /'eddy_wrapped_avg_image.nii.gz'
bvalsPath = data_dir / 'bvals'
bvalsPath_b9 = data_dir / 'bvals_b9'
bvecsPath = data_dir / 'bvecs'
ParcellationFsPath = data_dir / 'mri/aparc+aseg.mgz'
rACC_mod_aparc_aseg = data_dir / 'rACC_mod_aparc_aseg.nii.gz'
rACC_split_labels = {11026: data_dir / 'lh_rostralanteriorcingulate_ROI_acpc_1.nii.gz',
                     21026: data_dir / 'lh_rostralanteriorcingulate_ROI_acpc_2.nii.gz',
                    12026: data_dir / 'rh_rostralanteriorcingulate_ROI_acpc_1.nii.gz',
                     22026: data_dir / 'rh_rostralanteriorcingulate_ROI_acpc_2.nii.gz'}


# Freesurfer lookup table, e.g. https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT
lutPath = data_dir / 'FreesurferLookup.csv'

#tckPath=Path('/home/naxos2-raid25/sreta001/DBS_for_sreta001/DBS-OCD/OCD004/Code/app-track_aLIC_harelpreproc/output/track.tck')
saveFigDir = startDir / 'output'

to_check = [ parcellationPath, refT1Path, lutPath, diff_b0acpc_path, 
            diff_unreg_path, ParcellationFsPath, bvalsPath, bvalsPath_b9, bvecsPath]
for i in to_check:
    print(i)
    if not i.is_file():
        warn('%s doesn''t exist!' % str(i))

## run app-track_aLIC

In [ ]:
# save python variables for use by bash
os.environ['bvalsPath'] = str(bvalsPath)
os.environ['diffPath'] = str(diffPath)
os.environ['bvalsPath_b9'] = str(bvalsPath_b9)
os.environ['diff_b0acpc_path'] = str(diff_b0acpc_path)
os.environ['diff_unreg_path'] = str(diff_unreg_path)
os.environ['startDir'] = str(startDir)
#os.environ[''] = str()

In [ ]:
%%script bash
# Edit bvals file so that B0s have b-value 9 instead of 10

sed -e 's/^10/9/g' -e 's/ 10 / 9 /g' -e 's/10$/9/g' ${bvalsPath} > ${bvalsPath_b9}


In [ ]:
%%script bash
# copy affine from eddy_wrapped_B0_image_space-acpc.nii.gz to eddy_wrapped_avg_image_space-acpc.nii.gz

cp ${diff_unreg_path} ${diffPath}
fslcpgeom ${diff_b0acpc_path} ${diffPath} -d # -d option is needed to preserve the 4-d volume

In [ ]:
%%script bash

# link indata to app-track_aLIC/indata
ln -s "../indata" "${startDir}/app-track_aLIC/indata"

In [ ]:
%%script  bash
# invert acpc to mni xfm
convert_xfm -omat "indata/MNI2acpcLinear.mat" -inverse "indata/acpc2MNILinear.mat"

In [ ]:
%%script bash
# fails if ran on jupyter notebook, run in terminal instead
# run "main" app-track_aLIC script
export APPTAINER_BIND="/home,${APPTAINER_BIND}"

cd "${startDir}/app-track_aLIC" #navigate to startDir, then run ./main
./main

## Modify aparc+aseg with divided rACC ROI

In [ ]:
#load aparc+aseg in acpc
aparc_aseg = nib.load(parcellationPath)
aparc_aseg_voxel_grid = aparc_aseg.get_fdata()

#load rACC masks and voxel grid
for key, value in rACC_split_labels.items():
    rACC_ROI = nib.load(value)
    rACC_ROI_voxel_grid = rACC_ROI.get_fdata() #values either 0 and 1 because it's a mask
    aparc_aseg_voxel_grid[rACC_ROI_voxel_grid>0.5] = key #selecting aparc_aseg voxels within rACC mask and output corresponding key

aparc_aseg_nifti = nib.Nifti1Image(aparc_aseg_voxel_grid,aparc_aseg.affine) #convert to nifti image
nib.save(aparc_aseg_nifti,filename=rACC_mod_aparc_aseg)

## Subsegment Tracks

In [ ]:
targetLabels={'left':[1002,11026,21026,1012,1020,1028,1003,1014,1019,1027],
              'right':[2002,12026,22026,2012,2020,2028,2003,2014,2019,2027]}
spineLabels = {'left': [28, 16, 10], 
               'right': [16, 60, 49]}

#paths to input data

track_files = {
    'left': [startDir / 'app-track_aLIC' / 'output' / 'combined_aLIC_left.tck',],
    'right': [startDir / 'app-track_aLIC' / 'output' / 'combined_aLIC_right.tck',]}


In [ ]:
# sanity check inputs
to_check = [ parcellationPath, refT1Path, lutPath]
for side in ['left', 'right']:
    for tck in track_files[side]:
        to_check.append(tck)
        
for i in to_check:
    print(i)
    assert(i.is_file())

In [ ]:
# define functions to generate tck for target

def get_streams_matching_target(streams, atlas, target):
    target_mask=wmaPyTools.roiTools.multiROIrequestToMask(atlas,target)
    # return boolean mask for stream selection
    return wmaPyTools.segmentationTools.segmentTractMultiROI(streams, 
                    [target_mask,], 
                    [True,], 
                    ['either_end',]) 
    
def save_density_map(streams, ref_img, out_file):
    density=utils.density_map(streams, ref_img.affine, ref_img.shape)
    densityNifti = nib.nifti1.Nifti1Image(density, ref_img.affine, ref_img.header)
    nib.save(densityNifti, out_file)
    
def save_streams_matching_target(streams, atlas, lookupTable, target, out_file):
    strTarget = lookupTable.loc[target, 'LabelName:']
    print('target label is: %s (%s)' % (target, strTarget))
    #out_file = Path(save_dir) / ('track_%04d_%s' % (target,strTarget)) #no file extension yet, add it later
    print(out_file)
    # get boolean vector of matching streams
    targetBool = get_streams_matching_target(streams, atlas, target)
    streams = streams[targetBool]
    
    #dipy quickbundles
    streams = streams[bundle(streams)]
    
    #save *.tck tractogram
    wmaPyTools.streamlineTools.stubbornSaveTractogram(streams,
        savePath=str(out_file.with_suffix('.tck')))
    # save nifti density map
    save_density_map(streams, atlas, out_file.with_suffix('.nii.gz'))
    return targetBool
    
#targetBool = save_streams_matching_target(streams,inflatedAtlas, lookupTable, iTarget, saveFigDir)
#wmaPyTools.streamlineTools.stubbornSaveTractogram(streams[targetBool], 
#    savePath=str(saveFigDir / '1002_test.tck' )

In [ ]:
#apply the initial culling, to remove extraneous streamlines 
#first requires doing a DIPY quickbundling
def bundle(streams):
    print("DIPY quickbundle")
    clusters=wmaPyTools.streamlineTools.quickbundlesClusters(streams, thresholds = [30,20,10], nb_pts=100)

    #use those clusters to identify the streamlines to be culled
    print("identify streamlines to remove")
    survivingStreamsIndices, culledStreamIndicies=wmaPyTools.streamlineTools.cullViaClusters(clusters,streams,3)
    #convert survivingStreamsIndicies into a bool vec
    survivingStreamsBoolVec=np.zeros(len(streams),dtype=bool)
    survivingStreamsBoolVec[survivingStreamsIndices]=True
    
    print('%d of %d streams survived' % (len(survivingStreamsIndices), len(survivingStreamsBoolVec)))
    
    return survivingStreamsBoolVec

In [ ]:
# load atlas-based segmentation - modified rACC mask (Dan calls it a parcellation)
parcellaton=nib.load(rACC_mod_aparc_aseg)

In [ ]:
# load T1 anatomical image
refT1=nib.load(refT1Path)

In [ ]:
# load Freesurfer labels
lookupTable=pd.read_csv(lutPath,index_col='#No.')

In [ ]:
#perform inflate & deIsland of input parcellation
inflated_atlas_file = saveFigDir / Path(Path(rACC_mod_aparc_aseg.stem).stem + '_inflated').with_suffix('.nii.gz')
print(inflated_atlas_file)
inflatedAtlas,deIslandReport,inflationReport= wmaPyTools.roiTools.preProcParc(parcellaton,deIslandBool=True,inflateIter=2,retainOrigBorders=False,maintainIslandsLabels=None,erodeLabels=[2,41])    
nib.save(inflatedAtlas,filename=inflated_atlas_file)

In [ ]:
#Main cell, do all the hard work

for iSide in ['left', 'right']:
    for track_file in track_files[iSide]:        
        # load & orient streamlines
        
        tck_oriented_file = saveFigDir / Path(track_file.stem + '_oriented').with_suffix('.tck')
        if tck_oriented_file.exists():
            print('oriented tck already exists. loading %s' % tck_oriented_file)
            tckIn=nib.streamlines.load(tck_oriented_file)
            streams = tckIn.streamlines
        else:
            print('Load tck %s' % track_file)
            tckIn=nib.streamlines.load(track_file)
            print("orienting streamlines")
            streams=wmaPyTools.streamlineTools.orientAllStreamlines(tckIn.streamlines)
            # do quickbundles (never mind, takes too long)
            #streams = streams[bundle(streams)]
            # save oriented + bundled streams
            print('saving oriented tck %s' % tck_oriented_file)
            wmaPyTools.streamlineTools.stubbornSaveTractogram(streams,savePath=str(tck_oriented_file))
        
        parent_density_file = saveFigDir / Path(track_file.stem).with_suffix('.nii.gz')
        print('saving density map %s' % parent_density_file)
        save_density_map(streams, inflatedAtlas, parent_density_file)
        
        for iTarget in targetLabels[iSide]:
            targetStr = lookupTable.loc[iTarget, 'LabelName:']
            out_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr))
            print('Starting processing for %s' % out_file.stem)
            
            # subsegment the streams and save the resulting density map and tck tractogram
            targetBool = save_streams_matching_target(streams, inflatedAtlas, lookupTable, iTarget, out_file)

            

In [ ]:
# Calculate centroid for each pathway
APaxis = 1

for iSide in ['left', 'right']: #iterate over each hemisphere
    for track_file in track_files[iSide]: 
        for iTarget in targetLabels[iSide]: #iterate over each pathway
            targetStr = lookupTable.loc[iTarget, 'LabelName:'] #label corresponding to each pathway
            in_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr)) #generate input file
            in_nifti = nib.load(in_file.with_suffix('.nii.gz')) #load nifti of a pathway
            in_img = in_nifti.get_fdata() #convert nifti into a voxel array
            num_slices = np.shape(in_img)[APaxis]
            out_file = saveFigDir / ('%s_%04d_%s_centerofmass' % (track_file.stem, iTarget, targetStr)) #output center of mass image
            centerofmass = np.zeros([num_slices,3]) #numerical array corresponding to x,y,z coordinates of centroid
            for iSlice in range(num_slices): #interating over total number of coronal slides of input image
                img_slice = in_img[:,iSlice,:] #an individual slice
                tmp = ndimage.center_of_mass(img_slice, labels=None, index=None) #step that calculates COM for each slice
                centerofmass[iSlice,:] = [tmp[0],iSlice,tmp[1]] #adding iSlice at APaxis=1
            centerofmass = centerofmass[~np.any(np.isnan(centerofmass),axis=1)]
            nib.affines.apply_affine(in_nifti.affine, centerofmass, inplace=True)
            np.savetxt(out_file.with_suffix('.csv'), centerofmass, delimiter=",") #save output file as a .csv for all slices
            
                
            

In [ ]:
# Calculate (within ALIC) centroid for each pathway using ALIC mask
#lh_ALIC_mask = fullCutIC_ROI11_left.nii.gz
#rh_ALIC_mask = fullCutIC_ROI11_right.nii.gz

ALIC_mask_dir = {'left': data_dir / 'fullCutIC_ROI11_left.nii.gz',
                'right': data_dir/ 'fullCutIC_ROI11_right.nii.gz'}

APaxis = 1

for iSide in ['left', 'right']: #iterate over each hemisphere
    ALIC_mask = nib.load(ALIC_mask_dir[iSide])#loading ALIC mask
    for track_file in track_files[iSide]: 
        for iTarget in targetLabels[iSide]: #iterate over each pathway
            targetStr = lookupTable.loc[iTarget, 'LabelName:'] #label corresponding to each pathway
            in_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr)) #generate input file
            in_nifti = nib.load(in_file.with_suffix('.nii.gz')) #load nifti of a pathway
            in_img = in_nifti.get_fdata() #convert nifti into a voxel array
            resample_ALIC_mask = dipy.align.resample(ALIC_mask,in_nifti) #resample ALIC mask nifti into heatmap nifti dimensions
            in_img = in_img*resample_ALIC_mask.get_fdata() #multiply ALIC density map voxel array by resampled ALIC mask array
            num_slices = np.shape(in_img)[APaxis]
            out_file = saveFigDir / ('%s_%04d_%s_centerofmass_withinALIC' % (track_file.stem, iTarget, targetStr)) #output center of mass image
            centerofmass = np.zeros([num_slices,3]) #numerical array corresponding to x,y,z coordinates of centroid
            for iSlice in range(num_slices): #interating over total number of coronal slides of input image
                img_slice = in_img[:,iSlice,:] #an individual slice
                tmp = ndimage.center_of_mass(img_slice, labels=None, index=None) #step that calculates COM for each slice
                centerofmass[iSlice,:] = [tmp[0],iSlice,tmp[1]] #adding iSlice at APaxis=1
            centerofmass = centerofmass[~np.any(np.isnan(centerofmass),axis=1)]
            nib.affines.apply_affine(in_nifti.affine, centerofmass, inplace=True)
            np.savetxt(out_file.with_suffix('.csv'), centerofmass, delimiter=",") #save output file as a .csv for all slices
            

## Appendix

In [ ]:
# generate combined niftis which are the sum of all sides
for iSide in ['left', 'right']:
    for iTarget in targetLabels[iSide]:
        combined_img = np.zeros(refT1.shape)
        for track_file in track_files[iSide]:  # iterate over inferior and superior
            targetStr = lookupTable.loc[iTarget, 'LabelName:']
            in_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr))
            print(in_file)
            combined_img += nib.load(in_file.with_suffix('.nii.gz')).get_fdata()
        combined_file  = saveFigDir / ('combined_aLIC_%04d_%s' % ( iTarget, targetStr))
        combinedNifti = nib.nifti1.Nifti1Image(combined_img, refT1.affine, refT1.header)
        print(combined_file)
        nib.save(combinedNifti, combined_file.with_suffix('.nii.gz'))
        

In [ ]:
# load tractogram
# SKIPPED because we're iterating over multiple tractograms
tckIn=nib.streamlines.load(tckPath)

In [ ]:
# orient all the steamlines, potentially not necessary given redundancy with 
# subsequent steps
# SKIPPED because we're using either_end selection

print("orienting streamlines")
orientedStreams=wmaPyTools.streamlineTools.orientAllStreamlines(tckIn.streamlines)

In [ ]:
# save oriented streams
# SKIPPED
print("save oriented tck")
subTckSavePath=os.path.join(saveFigDir,'track_oriented.tck')
wmaPyTools.streamlineTools.stubbornSaveTractogram(orientedStreams,savePath=subTckSavePath)


In [ ]:
# save lite streamlines
#SKIPPED
n_streams_to_keep = int(5E5)
select_bool = np.random.choice(range(len(orientedStreams)), n_streams_to_keep, replace=False)
lite_streams = orientedStreams[select_bool]
wmaPyTools.streamlineTools.stubbornSaveTractogram(lite_streams,
    savePath=str(Path(saveFigDir) / 'track_lite.tck'))


In [ ]:
# select streamlines to work on

streams = lite_streams


In [ ]:
from dipy.tracking.utils import density_map
from wmaPyTools.visTools import multiTileDensity

multiTileDensity(streams,refT1,saveFigDir,'density',densityThreshold=0,noEmpties=True)

In [ ]:
M, grouping = utils.connectivity_matrix(streams, inflatedAtlas.affine, inflatedAtlas.get_fdata().astype(np.int),
                                        return_mapping=True,
                                        mapping_as_streamlines=True)

In [ ]:
np.shape(M)

In [ ]:
targets = targetLabels['left'] + targetLabels['right']
targets.sort()
for iTarget in targets:
    print(iTarget)
    print(lookupTable.loc[iTarget, 'LabelName:'])

In [ ]:
with open(bvalsPath, 'r') as f:
    x = f.readlines()

In [ ]:
#y = split(x, ' ')
y = x[0].split(' ')


In [ ]:
len(y)

In [ ]:
print(y)